In [1]:
import cymetric as cym
import os
import pandas as pd
from pandas.util.testing import assert_frame_equal
import numpy as np
from uuid import UUID
from cymetric.tools import raw_to_series, ensure_dt_bytes

%matplotlib inline 


/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
# Open the database
db = cym.dbopen("test-input.h5")
metadata = cym.root_metric(name='Metadata')

In [3]:
# Define an Evaluator, allows to acces the different data tables in the database
myEval = cym.Evaluator(db)

RuntimeError: basic_string

In [ ]:
# Acces on specific table (here Materials)
metadata = myEval.eval('Metadata')
time = myEval.eval('TimeList')
agent = myEval.eval('AgentEntry')

In [ ]:
deployment = metadata[ metadata['Type'] == "deployment" ]
dep_agent = agent[agent['AgentId'].isin(deployment['AgentId'])]

In [ ]:
dep_agent

In [27]:
metadata

,SimId,AgentId,keyword,Type,Value
0,97a733da-77ae-4dbe-b978-3226880f4ca3,16,co2,decommission,25.000000
1,97a733da-77ae-4dbe-b978-3226880f4ca3,16,co2,deployment,45.000000
2,97a733da-77ae-4dbe-b978-3226880f4ca3,16,co2,timestep,35.000000
3,97a733da-77ae-4dbe-b978-3226880f4ca3,16,co2,throughput,15.000000
4,97a733da-77ae-4dbe-b978-3226880f4ca3,16,land,decommission,24.000000
5,97a733da-77ae-4dbe-b978-3226880f4ca3,16,lolipop,throughput,11.000000
6,97a733da-77ae-4dbe-b978-3226880f4ca3,16,manpower,timestep,32.000000
7,97a733da-77ae-4dbe-b978-3226880f4ca3,16,water,deployment,43.000000
8,97a733da-77ae-4dbe-b978-3226880f4ca3,17,co2,decommission,25.000000
9,97a733da-77ae-4dbe-b978-3226880f4ca3,17,co2,deployment,45.000000


In [8]:
_tmp = pd.merge(deployment[['SimId', 'AgentId','keyword', 'Value']], dep_agent, on=['SimId', 'AgentId'])

In [9]:
deployment_use = pd.DataFrame(data={'SimId': _tmp.SimId, 'AgentId': _tmp.AgentId, 'Time': _tmp.EnterTime, 'keyword': _tmp.keyword, 'Value':_tmp.Value}, 
                              columns=['SimId', 'AgentId', 'Time', 'keyword', 'Value'])

In [10]:
deployment_use

,SimId,AgentId,Time,keyword,Value
0,97a733da-77ae-4dbe-b978-3226880f4ca3,16,2,co2,45.000000
1,97a733da-77ae-4dbe-b978-3226880f4ca3,16,2,water,43.000000
2,97a733da-77ae-4dbe-b978-3226880f4ca3,17,3,co2,45.000000
3,97a733da-77ae-4dbe-b978-3226880f4ca3,17,3,water,43.000000


In [11]:
decom = metadata[ metadata['Type'] == "decommission" ]
decom_agent = agent[agent['AgentId'].isin(decom['AgentId'])]
decom_agent['ExitTime'] = decom_agent['EnterTime'] + decom_agent['Lifetime']

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
decom_agent

,SimId,AgentId,Kind,Spec,Prototype,ParentId,Lifetime,EnterTime,ExitTime
5,97a733da-77ae-4dbe-b978-3226880f4ca3,16,Facility,:cycamore:Reactor,Reactor2,12,12,2,14
6,97a733da-77ae-4dbe-b978-3226880f4ca3,17,Facility,:cycamore:Reactor,Reactor3,12,15,3,18


In [13]:
_tmp = pd.merge(decom[['SimId', 'AgentId','keyword', 'Value']], decom_agent, on=['SimId', 'AgentId'])

In [14]:
decom_use = pd.DataFrame(data={'SimId': _tmp.SimId, 'AgentId': _tmp.AgentId, 'Time': _tmp.ExitTime, 'keyword': _tmp.keyword, 'Value':_tmp.Value}, 
                              columns=['SimId', 'AgentId', 'Time', 'keyword', 'Value'])

In [15]:
decom_use

,SimId,AgentId,Time,keyword,Value
0,97a733da-77ae-4dbe-b978-3226880f4ca3,16,14,co2,25.000000
1,97a733da-77ae-4dbe-b978-3226880f4ca3,16,14,land,24.000000
2,97a733da-77ae-4dbe-b978-3226880f4ca3,17,18,co2,25.000000
3,97a733da-77ae-4dbe-b978-3226880f4ca3,17,18,land,24.000000


In [16]:
timestep = metadata[ metadata['Type'] == "timestep" ]
timestep_agent = agent[agent['AgentId'].isin(timestep['AgentId'])]
timestep_agent['ExitTime'] = timestep_agent['EnterTime'] + timestep_agent['Lifetime']

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
timestep_agent

,SimId,AgentId,Kind,Spec,Prototype,ParentId,Lifetime,EnterTime,ExitTime
5,97a733da-77ae-4dbe-b978-3226880f4ca3,16,Facility,:cycamore:Reactor,Reactor2,12,12,2,14
6,97a733da-77ae-4dbe-b978-3226880f4ca3,17,Facility,:cycamore:Reactor,Reactor3,12,15,3,18


In [18]:
_tmp = pd.merge(timestep[['SimId', 'AgentId','keyword', 'Value']], timestep_agent, on=['SimId', 'AgentId'])

In [19]:
timestep_use = pd.DataFrame(data={'SimId': _tmp.SimId, 'AgentId': _tmp.AgentId, 'EnterTime': _tmp.EnterTime, 'ExitTime': _tmp.ExitTime , 'keyword': _tmp.keyword, 'Value':_tmp.Value}, 
                              columns=['SimId', 'AgentId', 'EnterTime', 'ExitTime', 'keyword', 'Value'])

In [20]:
data = []
for index, row  in timestep_use.iterrows():
    for i in range(row['EnterTime'], row['ExitTime']):
        data.append( (row['SimId'],
                      row['AgentId'],
                      i,
                      row['keyword'],
                      row['Value']))

timestep_use = pd.DataFrame(data, columns=['SimId', 'AgentId', 'Time', 'keyword', 'Value'])
print(timestep_use)

                                   SimId  AgentId  Time   keyword      Value
0   97a733da-77ae-4dbe-b978-3226880f4ca3       16     2       co2  35.000000
1   97a733da-77ae-4dbe-b978-3226880f4ca3       16     3       co2  35.000000
2   97a733da-77ae-4dbe-b978-3226880f4ca3       16     4       co2  35.000000
3   97a733da-77ae-4dbe-b978-3226880f4ca3       16     5       co2  35.000000
4   97a733da-77ae-4dbe-b978-3226880f4ca3       16     6       co2  35.000000
5   97a733da-77ae-4dbe-b978-3226880f4ca3       16     7       co2  35.000000
6   97a733da-77ae-4dbe-b978-3226880f4ca3       16     8       co2  35.000000
7   97a733da-77ae-4dbe-b978-3226880f4ca3       16     9       co2  35.000000
8   97a733da-77ae-4dbe-b978-3226880f4ca3       16    10       co2  35.000000
9   97a733da-77ae-4dbe-b978-3226880f4ca3       16    11       co2  35.000000
10  97a733da-77ae-4dbe-b978-3226880f4ca3       16    12       co2  35.000000
11  97a733da-77ae-4dbe-b978-3226880f4ca3       16    13       co2  35.000000